# Практическая работа №3: Обработка выборочных данных. Нахождение интервальных оценок параметров распределения. Проверка статистической гипотезы о нормальном распределении

Выполнили студенты гр. 9381 Шахин Никита и Андрух Ирина. Вариант №4

## Цель работы

Получение практических навыков вычисления интервальных статистических оценок параметров распределения выборочных данных и проверки «справедливости» статистических гипотез.

## Основные теоретические положения

Доверительным называют интервал, который с заданной надежностью $\gamma$ покрывает заданный параметр.

Интервальной оценкой математического ожидания по выборочной среднем $\bar{x}_B$ при неизвестном среднем квадратическом отклонении $\sigma$ генеральной совокупности служит доверительный интервал:
$$ \bar{x}_{\textit{в}} - \dfrac{S}{\sqrt{n}}t_{\gamma} \leqslant a \leqslant \bar{x}_\textit{в} + \dfrac{S}{\sqrt{n}}t_{\gamma} $$

где

$ \bar{x}_{\textit{в}} $ – статистическая оценка математического ожидания;

$ S $ – исправленная выборочная дисперсия;

$ n $ – объём выборки;

$ t_{\gamma} $ – из таблицы.

Интервальной оценкой среднеквадратического отклонения $\sigma$ по исправленной выборочной дисперсии служит доверительный интервал: 
$$S(1 - q) \leqslant \sigma \leqslant S(1 + q), \quad q < 1$$
$$0 \leqslant \sigma \leqslant S(1 + q), \quad q > 1$$

где

$ S $ – исправленная выборочная дисперсия; $ q $ – из таблицы.

Критерий Пирсона, или критерий $ \chi^2 $ (Хи-квадрат), применяют для проверки гипотезы о соответствии эмпирического распределения предполагаемому теоретическому распределению $ F(x) $.

Метод позволяет оценить статистическую значимость различий двух или нескольких относительных показателей (частот, долей).
Теоретические частоты вычисляются по формуле:
$$ n'_i = p_i \cdot n $$
где
$$ p_i = \frac{1}{\sigma\sqrt{2\pi}}\int\exp\Bigl(-\dfrac{(x-a)^2}{2\sigma^2}\Bigr)\,dx $$

Следует привести теоретические частоты к функции Лапласа. Если $ z = \dfrac{x - a}{\sigma} $, то интеграл примет следующий вид:
$$ \frac{1}{\sqrt{2\pi}}\int\exp\Bigl(-\dfrac{z^2}{2}\Bigr)\,dz $$

Для данной задачи $ z_i = \dfrac{x_i - \bar{x}_{\textit{в}}}{S}$. Преобразуя формулу $ p_i $, получим:
$$ p_i = \Phi(z_{i + 1}) - \Phi(z_i), $$
где
$$ \Phi(z_i) = \dfrac{1}{\sqrt{2\pi}} \int\limits_0^{z_i}\exp(x^2/2)\,dx $$

Если $\chi^2_{\textit{набл}} \leqslant \chi^2_{\textit{крит}}$ – гипотеза принимается, иначе гипотезу отвергают.

## Постановка задачи

Для заданной надежности определить (на основании выборочных данных и результатов выполнения практической работы №2) границы доверительных интервалов для математического ожидания и среднеквадратичного отклонения случайной величины. Проверить гипотезу о нормальном распределении исследуемой случайной величины с помощью критерия Пирсона $\chi^2$. Дать содержательную интерпретацию полученным результатам.

## Выполнение работы


Была вычислена точность и доверительный интервал для математического ожидания при неизвестном среднеквадратичном отклонении при заданном объёме выборки для доверительной точности $ \gamma $ равное 0.95, 0.99 и 0.999.

In [12]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

df = pd.read_csv('sample.csv', header=None).iloc[:, 0]

ranked_series = df.sort_values()

variation_series = ranked_series.apply(lambda x: sum(ranked_series == x))
relative_var_series = variation_series.apply(lambda x: x / len(df))

variation_df = pd.DataFrame({'Значение': ranked_series, 'Частота': variation_series,
                             'Относительная частота': relative_var_series}).drop_duplicates()


k = 1.33 + 3.31 * np.log10(len(df))
k = int(np.round(k, 0))
h = (max(ranked_series) - min(ranked_series)) / k
h = int(np.round(h, 0))

means = []
nums = []
relative_nums = []
distrib_nums = []
low_range = []
up_range = []
for i in range(min(ranked_series), max(ranked_series), h):
    cond = (i <= variation_df['Значение']) & \
           (variation_df['Значение'] < i + h) \
        if i + h < max(ranked_series) \
        else (i <= variation_df['Значение']) & (variation_df['Значение'] <= i + h)

    means.append((i * 2 + h) / 2)
    nums.append(variation_df['Частота'][cond].sum())
    low_range.append(i)
    up_range.append(i+h)

inter_df = pd.DataFrame({'Средние значения': means, 'Частота': nums}, dtype=np.int64)
C = inter_df.iloc[4, 0]
inter_df['Условные варианты'] = inter_df['Средние значения'].apply(lambda x: (x - C) / h)


for num_of_moment in range(1, 5):
    col = 'Условный момент {}'.format(num_of_moment)
    inter_df[col] = inter_df.iloc[:, 1:3].apply(lambda x: x[0]*(x[1]**num_of_moment), axis=1)

inter_df['Проверка'] = inter_df.iloc[:, 1:3].apply(lambda x: x[0]*((x[1]+1)**4), axis=1)

start_moment_1_emp = inter_df.iloc[:, :2].apply(lambda x: x[0]*x[1], axis=1).sum() / len(df)
central_moment_2_emp = inter_df.iloc[:, :2].apply(lambda x: ((x[0] - start_moment_1_emp)**2)*x[1], axis=1).sum() / len(df)

s = np.sqrt((len(df)/(len(df)-1)) * central_moment_2_emp)


print('Мат ожидание: ', start_moment_1_emp)
print('Дисперсия: ', central_moment_2_emp)

print('Среднекв отклонение: ', np.sqrt(central_moment_2_emp))

print('Исправленная выб. дисперсия: ', s)

print('n =', len(df))
t_95 = 1.983
t_99 = 2.623
t_999 = 3.386

sl = s * t_95 / np.sqrt(len(df))
i1 = start_moment_1_emp - sl
i2 = start_moment_1_emp + sl

print('Доверительный интервал для мат. ожидания. t = {} при y = 0.95  - ({}, {})'.format(t_95, i1, i2))
print('Доверительный интервал для мат. ожидания. t = {} при y = 0.99  - ({}, {})'.format(t_99, start_moment_1_emp - (s * t_99 / np.sqrt(len(df))), start_moment_1_emp + (s * t_99 / np.sqrt(len(df)))))
print('Доверительный интервал для мат. ожидания. t = {} при y = 0.999 - ({}, {})'.format(t_999, start_moment_1_emp - (s * t_999 / np.sqrt(len(df))), start_moment_1_emp + (s * t_999 / np.sqrt(len(df)))))

Мат ожидание:  449.1682242990654
Дисперсия:  3461.840859463708
Среднекв отклонение:  58.837410373534524
Исправленная выб. дисперсия:  59.11429383403294
n = 107
Доверительный интервал для мат. ожидания. t = 1.983 при y = 0.95  - (437.8357868306451, 460.5006617674857)
Доверительный интервал для мат. ожидания. t = 2.623 при y = 0.99  - (434.1783183587393, 464.1581302393915)
Доверительный интервал для мат. ожидания. t = 3.386 при y = 0.999 - (429.8179301648893, 468.5185184332415)


Из полученных результатов можно сделать вывод, что статистическая оценка математического ожидания попадет в интервалы вероятностью 95, 99 и 99.9%.

Были определены значения $ q $ при заданной доверительной надёжности $ \gamma $ для определения доверительных границ оценки среднеквадратичного отклонения $ \sigma $.

In [4]:
q_95 = 0.139
q_99 = 0.193
q_999 = 0.262

print('Доверительный интервал для среднекв. отклонения q = {} при y = 0.95  - ({}, {})'.format(q_95, s * (1 - q_95), s * (1 + q_95)))
print('Доверительный интервал для среднекв. отклонения q = {} при y = 0.99  - ({}, {})'.format(q_99, s * (1 - q_99), s * (1 + q_99)))
print('Доверительный интервал для среднекв. отклонения q = {} при y = 0.999 - ({}, {})'.format(q_999, s * (1 - q_999), s * (1 + q_999)))

Доверительный интервал для среднекв. отклонения q = 0.139 при y = 0.95  - (50.897406991102365, 67.33118067696353)
Доверительный интервал для среднекв. отклонения q = 0.193 при y = 0.99  - (47.70523512406458, 70.5233525440013)
Доверительный интервал для среднекв. отклонения q = 0.262 при y = 0.999 - (43.62634884951631, 74.60223881854958)


Из полученных результатов можно сделать вывод, что полученные интервалы покрывают величину $ \sigma $.


#### Проверить гипотезу о нормальности заданного распределения с помощью критерия Хи - квадрат (Пирсона). 

Вычислим теоретические вероятности и частоты попадания в каждый интервал.

Результаты представлены в таблицах

In [5]:
low_range.append(up_range[-1])
table = pd.DataFrame({'x': low_range})
table['$x-x_{mean}$'] = table['x'].apply(lambda x: x - start_moment_1_emp)
table['$v$'] = table['$x-x_{mean}$'].apply(lambda x: x / s)


def gauss_integral(z):
    return np.sqrt(np.pi) * math.erf(z / np.sqrt(2)) / np.sqrt(2)


def gauss(x):
    return np.exp(-np.power(x, 2.) / 2) / np.sqrt(2 * np.pi)


table['$\Phi(v)$'] = table['$v$'].apply(lambda x: gauss_integral(x) / np.sqrt(2 * np.pi))
display(table)

v_means = []
deltas = []
for i in range(0, len(table['$v$']) - 1):
    v_means.append((table['$v$'][i] + table['$v$'][i + 1]) / 2)
    deltas.append(table['$\Phi(v)$'][i + 1] - table['$\Phi(v)$'][i])

table2 = pd.DataFrame({'$v_{means}$': v_means, '$p_2$': deltas})
table2['$f(v_{mean})$'] = table2['$v_{means}$'].apply(gauss)
table2['$p_1$'] = table2['$f(v_{mean})$'].apply(lambda x: x * h / s)
table2['$n_1$'] = table2['$p_1$'].apply(lambda x: x * len(df))
table2['$n_2$'] = table2['$p_2$'].apply(lambda x: x * len(df))
display(table2)


,x,$x-x_{mean}$,$v$,$\Phi(v)$
0,320,-129.168224,-2.185059,-0.485558
1,354,-95.168224,-1.609902,-0.446290
2,388,-61.168224,-1.034745,-0.349606
3,422,-27.168224,-0.459588,-0.177094
4,456,6.831776,0.115569,0.046003
5,490,40.831776,0.690726,0.255131
6,524,74.831776,1.265883,0.397223
7,558,108.831776,1.841040,0.467192
8,592,142.831776,2.416197,0.492158
9,626,176.831776,2.991354,0.498611


,$v_{means}$,$p_2$,$f(v_{mean})$,$p_1$,$n_1$,$n_2$
0,-1.897481,0.039267,0.065930,0.037920,4.057479,4.201608
1,-1.322324,0.096684,0.166425,0.095721,10.242116,10.345228
2,-0.747167,0.172512,0.301777,0.173569,18.571890,18.458781
3,-0.172010,0.223097,0.393084,0.226085,24.191091,23.871373
4,0.403147,0.209128,0.367805,0.211546,22.635379,22.376717
5,0.978304,0.142091,0.247220,0.142190,15.214338,15.203779
6,1.553461,0.069970,0.119366,0.068654,7.346007,7.486752
7,2.128618,0.024966,0.041401,0.023812,2.547901,2.671369
8,2.703775,0.006453,0.010315,0.005933,0.634814,0.690479


Вычислим $\chi^2$  с использованием полученных частот по формуле:
$$\chi^2 = \sum\limits_{i = 1}^9 \dfrac{(nums-n)^2}{n}$$

In [6]:
sol1 = pd.DataFrame({'nums': nums, 'n': table2['$n_1$']})
sol1['$nums-n$'] = sol1.apply(lambda x: x[0]-x[1], axis=1)
sol1['$(nums-n)^2$'] = sol1['$nums-n$'].apply(lambda x: x**2)
sol1['$\dfrac{(nums-n)^2}{n}$']=sol1.apply(lambda x: x[3]/x[1], axis=1)
print('Solution 1 summ: ', sol1['$\dfrac{(nums-n)^2}{n}$'].sum())
display(sol1)

sol2 = pd.DataFrame({'nums': nums, 'n': table2['$n_2$']})
sol2['$nums-n$'] = sol2.apply(lambda x: x[0]-x[1], axis=1)
sol2['$(nums-n)^2$'] = sol2['$nums-n$'].apply(lambda x: x**2)
sol2['$\dfrac{(nums-n)^2}{n}$']=sol2.apply(lambda x: x[3]/x[1], axis=1)
print('Solution 2 summ: ', sol2['$\dfrac{(nums-n)^2}{n}$'].sum())
display(sol2)

Solution 1 summ:  11.650771283993393


,nums,n,$nums-n$,$(nums-n)^2$,$\dfrac{(nums-n)^2}{n}$
0,9,4.057479,4.942521,24.428515,6.020614
1,4,10.242116,-6.242116,38.964014,3.804293
2,23,18.571890,4.428110,19.608161,1.055798
3,22,24.191091,-2.191091,4.800880,0.198457
4,22,22.635379,-0.635379,0.403707,0.017835
5,17,15.214338,1.785662,3.188588,0.209578
6,7,7.346007,-0.346007,0.119721,0.016297
7,2,2.547901,-0.547901,0.300195,0.117821
8,1,0.634814,0.365186,0.133361,0.210078


Solution 2 summ:  11.193387654666825


,nums,n,$nums-n$,$(nums-n)^2$,$\dfrac{(nums-n)^2}{n}$
0,9,4.201608,4.798392,23.024570,5.479943
1,4,10.345228,-6.345228,40.261916,3.891835
2,23,18.458781,4.541219,20.622671,1.117228
3,22,23.871373,-1.871373,3.502037,0.146704
4,22,22.376717,-0.376717,0.141916,0.006342
5,17,15.203779,1.796221,3.226408,0.212211
6,7,7.486752,-0.486752,0.236928,0.031646
7,2,2.671369,-0.671369,0.450736,0.168729
8,1,0.690479,0.309521,0.095803,0.138749


$$\chi^2_1 = 11.65077$$
$$\chi^2_2 = 11.19338$$

$$\chi^2_{krit} = 11,070$$

Из полученных результатов можно сделать вывод, что данные отвергаются гипотезой $\chi^2$ и не имеют нормального распределения, так как $\chi^2 > \chi^2_{krit}$  в обоих способах.


## Выводы

В ходе выполнения практической работы были получены границы доверительных интервалов для математического ожидания и среднеквадратического отклонения случайной величины. Из полученных результатов можно сделать вывод, что интервал (437.8357868306451, 460.5006617674857) покрывает математическое ожидание  и интервал (50.66094981576623, 67.56763785229965) покрывает величину σ с вероятностью 95%.

интервал (434.1783183587393, 464.1581302393915) покрывает математическое ожидание  и интервал (47.70523512406458, 70.5233525440013) покрывает величину σ с вероятностью 99%.

интервал (429.8179301648893, 468.5185184332415) покрывает математическое ожидание  и интервал (43.62634884951631, 74.60223881854958) покрывает величину σ с вероятностью 99,9%.

Также была проверена гипотеза о нормальном распределении исследуемой случайной величины с помощью критерия Пирсона $\chi^2$. Из полученного результата можно сделать вывод, что гипотеза отвергается, т.к. $\chi^2 > \chi^2_{krit}$, соответственно, исследуемая случайная величина не принадлежит нормальному закону распределения. 